# AdminGPT: Your AI-Powered Administrative Assistant, powered by OpenAI's Assistant Framework  🚀
### Introduction
AdmiGPT is an AI-powered administrative assistant, harnessing the power of OpenAI's Assistant framework to seamlessly integrate with your email and calendar. Similar to Microsoft's Copilot, only better, it's designed to be your ultimate productivity partner, AdmiGPT offers an array of advanced features, making your administrative tasks simpler, faster, and more efficient.

AdminGPT is fully Open Source, so everything you need to run it for yourself is in this Github repo. This notebook helps you get started with AdminGPT, and walks you through how it's implemented. 

### Implementation
To begin, we're going to load our custom OpenAI Tools, which will interface with your email platform's API, and store any confidential and authentication information for the user in environmental variables.

In [1]:
from openai import OpenAI
import time, json, pprint, os
from tools.o365_toolkit import (
    o365search_emails,
    o365search_email,
    o365search_events,
    o365parse_proposed_times,
    o365send_message,
    o365send_event,
    tools,
)
from datetime import datetime as dt

# Set your name
os.environ["CLIENT_NAME"] = "Your Name"
# Set your email
os.environ["CLIENT_EMAIL"] = "Your Email"
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "Your OpenAI API Key"
# Set your Microsoft Graph client ID
os.environ["CLIENT_ID"] = "Your MS Graph Client ID"
# Set your Microsoft Graph client secret
os.environ["CLIENT_SECRET"] = "Your MS Graph Client Secret"

Next, we're going to set a few constants, which we will use throughout the code.

In [2]:
LOOP_DELAY_SECONDS = 2
debug = False
assistant_name = "Monica A. Ingenio"
model = "gpt-4-1106-preview"
current_date = dt.now()
formatted_date = current_date.strftime("%A, %B %d, %Y")
client_name = os.environ.get("CLIENT_NAME")
client_email = os.environ.get("CLIENT_EMAIL")
openai_api_key = os.environ.get("OPENAI_API_KEY")
assistant_instructions = (
    "You are an AI Administrative Assistant called "
    + assistant_name
    + ", and I am your executive. My name is "
    + client_name
    + ". My email is: "
    + client_email
    + ", in the Eastern Time (ET)."
    " You have access to my email and calendar. Today is "
    + formatted_date
    + ". "
)

# Add the debug prompt if user runs with debug
if debug:
    debug_prompt = (
        "Keep a record of any feedback requests provided by me"
        " detailing the prompt and tools calls in case I want to retrieve"
        " them."
    )
else:
    debug_prompt = ""    
assistant_instructions = assistant_instructions + debug_prompt

The next step is to create an OpenAI Assistant so we can interact with it, and a thread in which to run our prompts.

In [3]:
client = OpenAI(
        api_key=openai_api_key,
    )

assistant = client.beta.assistants.create(
    name="AI Administrative Assistant",
    instructions=assistant_instructions,
    model=model,
    tools=tools,
)

thread = client.beta.threads.create()

To make execution easier in the future steps, we create a function to run prompts with only one call. Then, we are going to use the function to submit the initial coaching prompt so that the Assistant gets better at performing tasks from the beginning. (A big part of the secret sauce of AdminGPT is in the coaching data file called coaching_data.txt)

In [4]:
def run_prompt(prompt: str, thread_id, assistant_id):
    message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=prompt,
    )

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run

with open("coaching_data.txt", "r") as file:
    prompt = file = file.read()

run = run_prompt(prompt, thread.id, assistant.id)

To further make execution easier in future steps, we Create a function that polls the OpenAI API for a response to the prompt and executes tool calls, so we can use it to retrieve responses in the future.

In [5]:
def poll_for_response(thread_id, run_id):

    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id, run_id=run_id
        )
        status = run.status

        if status == "completed":
            response = client.beta.threads.messages.list(thread_id=thread_id)
            if response.data:
                return(response.data[0].content[0].text.value)
            break
        elif status == "requires_action":
            tools_outputs = []

            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                function_arguments = tool_call.function.arguments
                function_arguments = json.loads(function_arguments)

                # Case statement to execute each toolkit function
                if function_name == "o365search_emails":
                    output = o365search_emails(**function_arguments)
                elif function_name == "o365search_email":
                    output = o365search_email(**function_arguments)
                elif function_name == "o365search_events":
                    output = o365search_events(**function_arguments)
                elif function_name == "o365parse_proposed_times":
                    output = o365parse_proposed_times(
                        **function_arguments, client=client, model=model
                    )
                elif function_name == "o365send_message":
                    output = o365send_message(**function_arguments)
                elif function_name == "o365send_event":
                    output = o365send_event(**function_arguments)

                # Clean the function output into JSON-like output
                output = pprint.pformat(output)
                tool_output = {"tool_call_id": tool_call_id, "output": output}
                tools_outputs.append(tool_output)

            if run.required_action.type == "submit_tool_outputs":
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run_id, tool_outputs=tools_outputs
                )

        elif status == "failed":
            return("Run failed try again!")
            break

        time.sleep(LOOP_DELAY_SECONDS)

Now that we've submit our first prompt, we are going to poll for a response. If the response is, "How can I help you?", we know that the coaching data was sent correctly, and we are ready to start using AdminGPT.

In [6]:
response = poll_for_response(thread.id, run.id)
print(response)

How can I help you?


To begin, we're going to perform the most simple task, which is summarizing an email from a specific sender. In this case, I chose a promotional email I received.

In [9]:
prompt = "Please summarize in a very short sentence the most recent email from Mark Hess"
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

Mark Hess shared Maven Associates' 2023 Q4 update, focusing on economic challenges and highlighting the company's growth and partnerships.


Now, we're going to respond to Mark's email, letting him know we're not interested, with AdminGPT drafting the content for us.

In [11]:
prompt = "Please draft a short email to Mark letting him know we're not interested. Please show me the email before sending."
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

Certainly, here's a draft of the email to Mark Hess:

---

Subject: RE: The Maven Newsletter - Winter 2023 Recap

Dear Mark,

Thank you for keeping us informed with the Maven Associates' quarterly update. We appreciate the insight, but currently, we will not be engaging further on this matter.

Warm regards,

Santiago Delgado

---

If this email looks good to you, please let me know, and I will send it.


I want to highlight a few things in this response because AdminGPT knew a lot of information with us telling them. First, it knew which Mark I was talking about, because it has access to this complete interaction. It also knew my name, which we sent as part of the original Assistant instructions. Finally, it knew how to bring it all together to write a courteous and clear email. This final step is when ChatGPT really shines by bringing different pieces of information together!